In [1]:
!pip install datasets
!pip install contractions
!pip install num2words
!pip install nltk
!pip install mlflow
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.3/143.3 kB 3.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 67.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 89.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.7/114.7 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 569.1/569.1 kB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 14.7 MB/s eta 0:00:00
  Attempting uninstall: cachetools
    Found existing installation: cachetools 4.2.4
    Uninstalling cachetools-4.2.4:
      Successfully uninstalled cachetools-4.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages tha

## Importing libraries


In [2]:
#1 importing libraries
from datasets import load_dataset
import pandas as pd
import re
from num2words import num2words
import contractions
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
import numpy as np
import evaluate
import mlflow
import mlflow.pytorch
import torch

In [3]:
import warnings
warnings.filterwarnings('ignore')

## Loading dataset

In [4]:
ds = load_dataset("ruslanmv/ai-medical-chatbot")

README.md:   0%|          | 0.00/863 [00:00<?, ?B/s]

dialogues.parquet:   0%|          | 0.00/142M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/256916 [00:00<?, ? examples/s]

In [5]:
# Iterate over each split and save as CSV
for split in ds.keys():
    df = pd.DataFrame(ds[split])  # Convert to DataFrame
    df.to_csv(f'ai doc_{split}.csv', index=False)

In [6]:
df

,Description,Patient,Doctor
0,Q. What does abutment of the nerve root mean?,"Hi doctor,I am just wondering what is abutting...",Hi. I have gone through your query with dilige...
1,Q. What should I do to reduce my weight gained...,"Hi doctor, I am a 22-year-old female who was d...",Hi. You have really done well with the hypothy...
2,Q. I have started to get lots of acne on my fa...,Hi doctor! I used to have clear skin but since...,Hi there Acne has multifactorial etiology. Onl...
3,Q. Why do I have uncomfortable feeling between...,"Hello doctor,I am having an uncomfortable feel...",Hello. The popping and discomfort what you fel...
4,Q. My symptoms after intercourse threatns me e...,"Hello doctor,Before two years had sex with a c...",Hello. The HIV test uses a finger prick blood ...
...,...,...,...
256911,Why is hair fall increasing while using Bontre...,I am suffering from excessive hairfall. My doc...,"Hello Dear Thanks for writing to us, we are he..."
256912,Why was I asked to discontinue Androanagen whi...,"Hi Doctor, I have been having severe hair fall...","hello, hair4u is combination of minoxid..."
256913,Can Mintop 5% Lotion be used by women for seve...,Hi..i hav sever hair loss problem so consulted...,HI I have evaluated your query thoroughly you...
256914,Is Minoxin 5% lotion advisable instead of Foli...,"Hi, i am 25 year old girl, i am having massive...",Hello and Welcome to ‘Ask A Doctor’ service.I ...


## Take a random sample of 300 rows


In [7]:
# Step 2: Take a random sample of 100 rows
sample_df = df.sample(n=300, random_state=1)  # random_state for reproducibility

# Step 3: Save the sampled data to a new CSV file
sample_df.to_csv('sampled_data.csv', index=False)

print("Sampled data saved to 'sampled_data.csv'")


Sampled data saved to 'sampled_data.csv'


## Cleaning Dataset

In [8]:
for column in df.columns:
    sample_df[column] = sample_df[column].str.lower()
df.head()

,Description,Patient,Doctor
0,Q. What does abutment of the nerve root mean?,"Hi doctor,I am just wondering what is abutting...",Hi. I have gone through your query with dilige...
1,Q. What should I do to reduce my weight gained...,"Hi doctor, I am a 22-year-old female who was d...",Hi. You have really done well with the hypothy...
2,Q. I have started to get lots of acne on my fa...,Hi doctor! I used to have clear skin but since...,Hi there Acne has multifactorial etiology. Onl...
3,Q. Why do I have uncomfortable feeling between...,"Hello doctor,I am having an uncomfortable feel...",Hello. The popping and discomfort what you fel...
4,Q. My symptoms after intercourse threatns me e...,"Hello doctor,Before two years had sex with a c...",Hello. The HIV test uses a finger prick blood ...


In [9]:
print(sample_df.isnull().sum())

Description    0
Patient        0
Doctor         0
dtype: int64


In [10]:
# Replace numbers with words
for column in sample_df.columns:
    sample_df[column] = sample_df[column].apply(lambda x: re.sub(r'\d+', lambda m: num2words(int(m.group())), x))  
# Remove 'Q' from the 'text_column'
# Remove 'Q' from the 'Description_tokens' column
sample_df['Description'] = sample_df['Description'].str.replace("q", '', regex=False)

# Remove both 'hi' and 'hello' from 'Patient_tokens' and 'Doctor_tokens'
sample_df['Patient'] = sample_df['Patient'].str.replace('hi', '', regex=False).str.replace('hello', '', regex=False)
sample_df['Doctor'] = sample_df['Doctor'].str.replace('hi', '', regex=False).str.replace('hello', '', regex=False)

# Print the updated DataFrame
print(sample_df)


                                              Description  \
1736    . why do periods get delayed after first time ...   
168192           what causes swelling on tongue and gums?   
23659   . is it possible for skin tags developed from ...   
75342   suggest medicnes for itchy lips after drinking...   
151776  what causes watery eyes and fever in a two yea...   
...                                                   ...   
248469  what is the cure for blocked fallopian tube ot...   
146900  what causes high heart rate with fever in a fi...   
147569  what causes vaginal itching and pain in ten ye...   
144383            suggest remedy for migraine in children   
160695     what causes dark smelly stools when on nanpro?   

                                                  Patient  \
1736     doctor, i had sex for the first time. after t...   
168192  for a couple days i've noticed an increasingly...   
23659    doctor, i have two small anal skin tags devel...   
75342   yes, just wonde

In [11]:
# Function to expand contractions using the contractions library
def expand_contractions(text):
    if isinstance(text, str):  # Ensure the value is a string
        return contractions.fix(text)
    return text

# Apply contraction expansion to all text columns
for column in df.columns:
    sample_df[column] = sample_df[column].apply(expand_contractions)

# View the DataFrame after expanding contractions
print(sample_df)


                                              Description  \
1736    . why do periods get delayed after first time ...   
168192           what causes swelling on tongue and gums?   
23659   . is it possible for skin tags developed from ...   
75342   suggest medicnes for itchy lips after drinking...   
151776  what causes watery eyes and fever in a two yea...   
...                                                   ...   
248469  what is the cure for blocked fallopian tube ot...   
146900  what causes high heart rate with fever in a fi...   
147569  what causes vaginal itching and pain in ten ye...   
144383            suggest remedy for migraine in children   
160695     what causes dark smelly stools when on nanpro?   

                                                  Patient  \
1736     doctor, i had sex for the first time. after t...   
168192  for a couple days i have noticed an increasing...   
23659    doctor, i have two small anal skin tags devel...   
75342   yes, just wonde

In [12]:

for column in df.columns:
    sample_df[column] = sample_df[column].apply(lambda x: re.sub(r'[^\w\s]', ' ', str(x)) if isinstance(x, str) else x)
    sample_df[column] = sample_df[column].apply(lambda x: re.sub(r'\s+', ' ', str(x)) if isinstance(x, str) else x)
    sample_df[column] = sample_df[column].apply(lambda x: x.strip() if isinstance(x, str) else x)
df.head()

,Description,Patient,Doctor
0,Q. What does abutment of the nerve root mean?,"Hi doctor,I am just wondering what is abutting...",Hi. I have gone through your query with dilige...
1,Q. What should I do to reduce my weight gained...,"Hi doctor, I am a 22-year-old female who was d...",Hi. You have really done well with the hypothy...
2,Q. I have started to get lots of acne on my fa...,Hi doctor! I used to have clear skin but since...,Hi there Acne has multifactorial etiology. Onl...
3,Q. Why do I have uncomfortable feeling between...,"Hello doctor,I am having an uncomfortable feel...",Hello. The popping and discomfort what you fel...
4,Q. My symptoms after intercourse threatns me e...,"Hello doctor,Before two years had sex with a c...",Hello. The HIV test uses a finger prick blood ...


## Data Preprocessing

In [13]:
nltk.download('punkt')
from nltk.tokenize import word_tokenize
sample_df['Description_tokens'] = sample_df['Description'].apply(lambda x: word_tokenize(x))
sample_df['Patient_tokens'] = sample_df['Patient'].apply(lambda x: word_tokenize(x))
sample_df['Doctor_tokens'] = sample_df['Doctor'].apply(lambda x: word_tokenize(x))

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [14]:
sample_df.head()

,Description,Patient,Doctor,Description_tokens,Patient_tokens,Doctor_tokens
1736,why do periods get delayed after first time sex,doctor i had sex for the first time after that...,as the serum beta hcg levels are less than thr...,"[why, do, periods, get, delayed, after, first,...","[doctor, i, had, sex, for, the, first, time, a...","[as, the, serum, beta, hcg, levels, are, less,..."
168192,what causes swelling on tongue and gums,for a couple days i have noticed an increasing...,thank you for consulting with healthcaremagic ...,"[what, causes, swelling, on, tongue, and, gums]","[for, a, couple, days, i, have, noticed, an, i...","[thank, you, for, consulting, with, healthcare..."
23659,is it possible for skin tags developed from he...,doctor i have two small anal skin tags develop...,i will definitely help you regarding your quer...,"[is, it, possible, for, skin, tags, developed,...","[doctor, i, have, two, small, anal, skin, tags...","[i, will, definitely, help, you, regarding, yo..."
75342,suggest medicnes for itchy lips after drinking...,yes just wondering if i took an allergy pill b...,from you story it is clear that you are suffer...,"[suggest, medicnes, for, itchy, lips, after, d...","[yes, just, wondering, if, i, took, an, allerg...","[from, you, story, it, is, clear, that, you, a..."
151776,what causes watery eyes and fever in a two yea...,my two yr old has been suffering from watery e...,i just read through your question in the major...,"[what, causes, watery, eyes, and, fever, in, a...","[my, two, yr, old, has, been, suffering, from,...","[i, just, read, through, your, question, in, t..."


In [15]:
#removing stop words
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))
for column in sample_df.columns[3:]:
    sample_df[column] = sample_df[column].apply(lambda x: [word for word in x if word not in stop_words])
df.head()

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,Description,Patient,Doctor
0,Q. What does abutment of the nerve root mean?,"Hi doctor,I am just wondering what is abutting...",Hi. I have gone through your query with dilige...
1,Q. What should I do to reduce my weight gained...,"Hi doctor, I am a 22-year-old female who was d...",Hi. You have really done well with the hypothy...
2,Q. I have started to get lots of acne on my fa...,Hi doctor! I used to have clear skin but since...,Hi there Acne has multifactorial etiology. Onl...
3,Q. Why do I have uncomfortable feeling between...,"Hello doctor,I am having an uncomfortable feel...",Hello. The popping and discomfort what you fel...
4,Q. My symptoms after intercourse threatns me e...,"Hello doctor,Before two years had sex with a c...",Hello. The HIV test uses a finger prick blood ...


In [16]:
nltk.download('wordnet')  # WordNet corpus for lemmatization
nltk.download('omw-1.4')  # Additional WordNet data

# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

# Function to lemmatize a sentence without tokenizing
def lemmatize_text(text):
    if isinstance(text, str):  # Ensure that the input is a string
        # Split text into words, lemmatize each word, and join them back into a sentence
        lemmatized_text = ([lemmatizer.lemmatize(word) for word in text.split()])
        return lemmatized_text
    else:
        return text  # Return the original value if it's not a string (e.g., NaN, int)

# Apply lemmatization to each column
for column in sample_df.columns[3:]:
    sample_df[column] = sample_df[column].apply(lambda x: lemmatize_text(x))




[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...


In [17]:
sample_df.head()

,Description,Patient,Doctor,Description_tokens,Patient_tokens,Doctor_tokens
1736,why do periods get delayed after first time sex,doctor i had sex for the first time after that...,as the serum beta hcg levels are less than thr...,"[periods, get, delayed, first, time, sex]","[doctor, sex, first, time, get, periods, yet, ...","[serum, beta, hcg, levels, less, three, miu, m..."
168192,what causes swelling on tongue and gums,for a couple days i have noticed an increasing...,thank you for consulting with healthcaremagic ...,"[causes, swelling, tongue, gums]","[couple, days, noticed, increasingly, painful,...","[thank, consulting, healthcaremagic, allergic,..."
23659,is it possible for skin tags developed from he...,doctor i have two small anal skin tags develop...,i will definitely help you regarding your quer...,"[possible, skin, tags, developed, healed, fiss...","[doctor, two, small, anal, skin, tags, develop...","[definitely, help, regarding, query, skin, tag..."
75342,suggest medicnes for itchy lips after drinking...,yes just wondering if i took an allergy pill b...,from you story it is clear that you are suffer...,"[suggest, medicnes, itchy, lips, drinking, alc...","[yes, wondering, took, allergy, pill, couple, ...","[story, clear, suffering, allergic, condition,..."
151776,what causes watery eyes and fever in a two yea...,my two yr old has been suffering from watery e...,i just read through your question in the major...,"[causes, watery, eyes, fever, two, year, old]","[two, yr, old, suffering, watery, eyes, fever,...","[read, question, majority, cases, ts, viral, i..."


In [18]:
df_prep = sample_df.drop(['Description','Patient','Doctor'],axis =1)
df_prep.rename(columns={'Description_tokens': 'Description', 'Patient_tokens': 'Patient', 'Doctor_tokens': 'Doctor'}, inplace=True)
df_prep.head()

,Description,Patient,Doctor
1736,"[periods, get, delayed, first, time, sex]","[doctor, sex, first, time, get, periods, yet, ...","[serum, beta, hcg, levels, less, three, miu, m..."
168192,"[causes, swelling, tongue, gums]","[couple, days, noticed, increasingly, painful,...","[thank, consulting, healthcaremagic, allergic,..."
23659,"[possible, skin, tags, developed, healed, fiss...","[doctor, two, small, anal, skin, tags, develop...","[definitely, help, regarding, query, skin, tag..."
75342,"[suggest, medicnes, itchy, lips, drinking, alc...","[yes, wondering, took, allergy, pill, couple, ...","[story, clear, suffering, allergic, condition,..."
151776,"[causes, watery, eyes, fever, two, year, old]","[two, yr, old, suffering, watery, eyes, fever,...","[read, question, majority, cases, ts, viral, i..."


In [19]:
df_prep.reset_index(drop=True, inplace=True)
df_prep

,Description,Patient,Doctor
0,"[periods, get, delayed, first, time, sex]","[doctor, sex, first, time, get, periods, yet, ...","[serum, beta, hcg, levels, less, three, miu, m..."
1,"[causes, swelling, tongue, gums]","[couple, days, noticed, increasingly, painful,...","[thank, consulting, healthcaremagic, allergic,..."
2,"[possible, skin, tags, developed, healed, fiss...","[doctor, two, small, anal, skin, tags, develop...","[definitely, help, regarding, query, skin, tag..."
3,"[suggest, medicnes, itchy, lips, drinking, alc...","[yes, wondering, took, allergy, pill, couple, ...","[story, clear, suffering, allergic, condition,..."
4,"[causes, watery, eyes, fever, two, year, old]","[two, yr, old, suffering, watery, eyes, fever,...","[read, question, majority, cases, ts, viral, i..."
...,...,...,...
295,"[cure, blocked, fallopian, tube, surgery]","[dear, sir, iam, trty, yrs, old, got, married,...","[saying, left, tube, blocked, tnk, right, tube..."
296,"[causes, high, heart, rate, fever, five, year,...","[five, year, old, daughter, born, alarge, vsd,...","[gh, heart, rates, expected, gh, fever, five, ..."
297,"[causes, vaginal, itching, pain, ten, year, old]","[daughter, suffers, frequently, vaginal, itcng...","[vaginal, itcng, girls, might, due, urinary, t..."
298,"[suggest, remedy, migraine, children]","[son, enzo, nine, years, old, recently, starte...","[headaches, common, among, cldren, since, head..."


In [20]:
for column in df_prep.columns:
    df_prep[column] = df_prep[column].apply(lambda x: ' '.join(x))

In [21]:
df_prep.head()

,Description,Patient,Doctor
0,periods get delayed first time sex,doctor sex first time get periods yet went gyn...,serum beta hcg levels less three miu ml pregna...
1,causes swelling tongue gums,couple days noticed increasingly painful happe...,thank consulting healthcaremagic allergic reac...
2,possible skin tags developed healed fissures r...,doctor two small anal skin tags developed heal...,definitely help regarding query skin tags form...
3,suggest medicnes itchy lips drinking alcohol,yes wondering took allergy pill couple drinks ...,story clear suffering allergic condition parti...
4,causes watery eyes fever two year old,two yr old suffering watery eyes fever two wee...,read question majority cases ts viral illness ...


## Preparing Data as input for the model

In [22]:
# Combine Patient and Description to create input for the model
df_prep['input_text'] = df_prep['Patient'] + " " + df_prep['Description'] + " [SEP]"
df_prep['target_text'] = df_prep['Doctor']


In [23]:
# Split the dataset into training and testing sets
train_df, test_df = train_test_split(df_prep, test_size=0.2, random_state=42)

## DialoGPT Model


In [24]:
# load model and tokenizer
# Load DialoGPT
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
tokenizer.pad_token = tokenizer.eos_token
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Load the model and move it to the appropriate device
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium").to(device)

tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/863M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
# Set the MLflow experiment name
mlflow.set_experiment("Chatbot_Model_Experiment")

# Custom Dataset class
class ChatbotDataset(Dataset):
    def __init__(self, inputs, targets, tokenizer, max_length):
        self.inputs = inputs
        self.targets = targets
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        input_text = self.inputs[idx]
        target_text = self.targets[idx]

        # Encode input and target
        input_encoding = self.tokenizer.encode_plus(
            input_text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        target_encoding = self.tokenizer.encode_plus(
            target_text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': input_encoding['input_ids'].flatten(),
            'attention_mask': input_encoding['attention_mask'].flatten(),
            'labels': target_encoding['input_ids'].flatten()
        }

In [26]:
# Set maximum length, batch size, and training arguments
MAX_LENGTH = 128
BATCH_SIZE = 2
EPOCHS = 15

# Prepare datasets
train_dataset = ChatbotDataset(
    inputs=train_df['input_text'].tolist(),
    targets=train_df['target_text'].tolist(),
    tokenizer=tokenizer,
    max_length=MAX_LENGTH
)

test_dataset = ChatbotDataset(
    inputs=test_df['input_text'].tolist(),
    targets=test_df['target_text'].tolist(),
    tokenizer=tokenizer,
    max_length=MAX_LENGTH
)


In [27]:
# Set up DataLoaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

## Train the model

In [28]:
#Training Arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    weight_decay=0.01,
    logging_dir='./logs',
    evaluation_strategy="steps", 
    save_strategy="steps",
    logging_strategy="steps",  
    logging_steps=1000
)
   
# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [29]:
# Train the model
with mlflow.start_run(run_name="Training Run"):
    trainer.train()
#wandb key :6f70c2ba3ec256ca6c9a7207a98ab3319899e52e    

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss
1000,3.124800,5.016912


## save the model

In [30]:
#Save the model
model.save_pretrained('./chatbot_model')
tokenizer.save_pretrained('./chatbot_model')


('./chatbot_model/tokenizer_config.json',
 './chatbot_model/special_tokens_map.json',
 './chatbot_model/vocab.json',
 './chatbot_model/merges.txt',
 './chatbot_model/added_tokens.json',
 './chatbot_model/tokenizer.json')

In [ ]:
# Log the model to MLflow
mlflow.pytorch.log_model(model, "model")

# End the MLflow run
mlflow.end_run()


## Responce generation

In [32]:

# Responce generation

def generate_response(input_text):
    with mlflow.start_run(nested=True):  # Nested run for logging responses
        # Log input text and parameters
        mlflow.log_param("input_text", input_text)
        mlflow.log_param("max_length", 150)
        mlflow.log_param("top_k", 50)
        mlflow.log_param("top_p", 0.95)
        mlflow.log_param("temperature", 0.7)

        # Encode the input text with the tokenizer
        input_ids = tokenizer.encode(input_text + tokenizer.eos_token, return_tensors='pt').to(device)

        #print(f"Input Text: {input_text}")
        #print(f"Tokenized Input IDs: {input_ids}")

        # Generate a response
        try:
            response_ids = model.generate(
                input_ids,
                max_length=150,
                num_return_sequences=1,
                pad_token_id=tokenizer.eos_token_id,
                do_sample=True,
                top_k=50,
                top_p=0.95,
                temperature=0.7
            )

            # Decode the generated response
            response = tokenizer.decode(response_ids[:, input_ids.shape[-1]:][0], skip_special_tokens=True)

            #print(f"Generated Response Token IDs: {response_ids}")
            #print(f"Decoded Response: {response}")

            # Log generated response
            mlflow.log_param("generated_response_token_ids", response_ids.tolist())
            mlflow.log_text(response, "generated_response.txt")

            # Check if the response is empty
            if response.strip() == "":
                print("Warning: Empty response generated.")
                response = "I'm sorry, I didn't understand that. Could you rephrase?"

            # Log the final response
            mlflow.log_param("final_response", response)
            return response

        except Exception as e:
            error_message = f"Error generating response: {e}"
            print(error_message)
            mlflow.log_param("error", error_message)
            return "An error occurred while generating a response."

## Test examples

In [33]:
# Example usage of the chatbot
user_input = "What are the possible causes of lower back pain?"
response = generate_response(user_input)
print("Chatbot Response:", response)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Chatbot Response: There are many possibilities including a diagnosis based on ank experience levelsofap based therapy routine pos exercises dynamic poses exercises dynamic poses regularity w exercises dynamic poses dynamic poses exercises multites care mult treatment people different care gai therapy


In [34]:
# Example usage of the chatbot
user_input = "What does abutment of the nerve root mean? What treatment is required for annular bulging and tear?"
response = generate_response(user_input)
print("Chatbot Response:", response)


Chatbot Response: An example would be visible buladen cosmetic surgery would be an example


In [35]:
torch.cuda.empty_cache()

In [36]:
# Function to interact with the chatbot
def chat_with_bot():
    print("Start chatting with the bot! Type 'quit' to stop.")
    
    while True:
        user_input = input("You: ")  # Get input from the user
        if user_input.lower() in ['exit', 'quit', 'bye']:  # Exit condition
            print("Chatbot: Goodbye!")
            break
        
        # Generate the chatbot's response using your function
        response = generate_response(user_input)
        
        # Print the chatbot's response
        print(f"Chatbot: {response}")

# Start the chatbot interaction
chat_with_bot()


Start chatting with the bot! Type 'quit' to stop.


You:  hi


Chatbot: Hi! How are you today?


You:  i am good 


Chatbot: hey good question please let me know if u find an answer


You:  bye


Chatbot: Goodbye!


In [43]:
patient_text = "I have been feeling fatigued for the past week and experiencing shortness of breath."
response = generate_response(patient_text)
print("Doctor's Response:", response)

Doctor's Response: Have you tried taking magnesium oxide tablets?


## Prompt


In [41]:
# Define the assistant persona and one-shot example
assistant_persona = """
You are a health assistant providing friendly, supportive, and general health advice to users.
You don’t diagnose, but you give practical tips and encourage consulting a doctor for specific concerns.
Use a conversational, empathetic tone, and keep responses clear and concise.
"""

# One-shot example to guide the model’s responses
one_shot_example = """
User: I have a headache and feeling a bit unwell.
Assistant: I'm sorry to hear that. Make sure to stay hydrated and rest. If your headache persists, consider reaching out to a healthcare provider for advice.

User: I have been feeling fatigued for the past week and experiencing shortness of breath.
Assistant: I'm sorry to hear that you've been feeling this way. Fatigue and shortness of breath can have a variety of causes, from lack of rest or stress to health conditions that may need attention. It could be helpful to ensure you're getting enough rest, staying hydrated, and managing stress. However, if these symptoms continue, it’s best to reach out to a healthcare professional for further evaluation.

User: What are the possible causes of lower back pain?
Assistant: Lower back pain is a common issue and can arise from muscle strain, poor posture, or even extended periods of sitting. Sometimes, conditions like herniated discs or arthritis can also be factors. Trying gentle stretches, maintaining good posture, and avoiding heavy lifting may help ease the pain. If the pain is persistent, consider consulting a doctor for a more thorough examination.

User: What does abutment of the nerve root mean? What treatment is required for annular bulging and tear?
Assistant: Abutment of the nerve root generally refers to when a structure, like a disc, is pressing close to a nerve, which can sometimes cause pain or discomfort. Annular bulging and tears can also contribute to similar symptoms. Treatments may vary depending on the severity, but they can include physical therapy, medications, or in some cases, surgical options. Consulting with a specialist would be ideal to understand the best course of action based on your specific situation.
"""


# Define a custom prompt template for each user message
def create_custom_prompt(input_text, persona, one_shot_example, previous_context=None):
    """
    Generates a structured prompt based on input, including persona, one-shot example, and chat history.
    """
    prompt = f"{persona}\n{one_shot_example}\n"
    
    # Add previous context if available
    if previous_context:
        prompt += previous_context + "\n"

    # Add the current user input to the prompt
    prompt += f"User: {input_text}\nAssistant:"
    return prompt

def get_chatbot_response(input_text):
    """Generates a response based on user input, structured prompting, and one-shot example."""
    global chat_history_ids

    # First, check for a scripted response to simple phrases
    scripted_response = generate_scripted_response(input_text)
    if scripted_response:
        return scripted_response

    # Create the custom prompt with persona, one-shot example, and chat history
    previous_context = tokenizer.decode(chat_history_ids) if chat_history_ids.shape[0] > 0 else ""
    prompt_text = create_custom_prompt(input_text, assistant_persona, one_shot_example, previous_context)
    
    # Encode the prompt and move to the same device as the model
    new_user_input_ids = tokenizer.encode(prompt_text + tokenizer.eos_token, return_tensors='pt').to(device)

    # Append new user input to chat history
    if chat_history_ids.shape[0] == 0:
        chat_history_ids = new_user_input_ids
    else:
        chat_history_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1).to(device)

    # Generate model response with tuned parameters
    response_ids = model.generate(
        chat_history_ids,
        max_length=chat_history_ids.shape[-1] + 100,
        pad_token_id=tokenizer.eos_token_id,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.15
    )

    # Decode the model response, skip past user input in history
    bot_response = tokenizer.decode(response_ids[:, chat_history_ids.shape[-1]:][0], skip_special_tokens=True)
    
    # Update chat history with model response
    chat_history_ids = torch.cat([chat_history_ids, response_ids[:, chat_history_ids.shape[-1]:]], dim=-1)
    
    return bot_response

# Example usage
print("Welcome to the Health Chatbot! How can I assist you today?")

while True:
    user_input = input("You: ")
    
    if user_input.lower() in ['exit', 'quit', 'bye']:
        print("Chatbot: Thank you for chatting! Stay healthy!")
        break

    # Generate a response
    response = get_chatbot_response(user_input)
    print("Chatbot:", response)


Welcome to the Health Chatbot! How can I assist you today?


You:  hi


Chatbot: Hello! How can I assist you today?


You:  can i ask you a question?


Chatbot: Of course! What would you like to ask?


...

Chatbot: You're welcome! Feel free to reach out if you need more help.


You:  good bye


Chatbot: Thank you for chatting! Take care and stay healthy.


You:  quit


Chatbot: Thank you for chatting! Stay healthy!


## MLOps(mlflow)

In [ ]:
!pip install pyngrok

In [ ]:
!mlflow ui --port 5000

In [ ]:
from pyngrok import ngrok

# Authtoken of ngrok = 2oR3aHtesovZ1ptvamSKDlqwsoQ_6mwSRkhrTpaVqTQpNkhfV
ngrok.set_auth_token("2oR3aHtesovZ1ptvamSKDlqwsoQ_6mwSRkhrTpaVqTQpNkhfV")

public_url = ngrok.connect(5000)
print(f"MLflow UI is available at {public_url}")